In [2]:
import tensorflow as tf

In [3]:
num_gpus_available = len(tf.config.experimental.list_physical_devices('GPU'))
print("Num GPUs Available: ", num_gpus_available)
assert num_gpus_available > 0


Num GPUs Available:  1


In [4]:
!pip install transformers

In [5]:
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
import pandas as pd
import numpy as np

In [6]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
from google.colab import drive

In [8]:
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
df = pd.read_csv('/content/drive/MyDrive/amazon product review/amazon_reviews_us_Mobile_Electronics_v1_00 (1).tsv', sep='\t', quoting=3)  # sep='\t' specifies tab as the delimiter, quoting=3 tells pandas to ignore quotes

In [10]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,20422322,R8MEA6IGAHO0B,B00MC4CED8,217304173,BlackVue DR600GW-PMP,Mobile_Electronics,5,0,0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31
1,US,40835037,R31LOQ8JGLPRLK,B00OQMFG1Q,137313254,GENSSI GSM / GPS Two Way Smart Phone Car Alarm...,Mobile_Electronics,5,0,1,N,Y,five star,it's great,2015-08-31
2,US,51469641,R2Y0MM9YE6OP3P,B00QERR5CY,82850235,iXCC Multi pack Lightning cable,Mobile_Electronics,5,0,0,N,Y,great cables,These work great and fit my life proof case fo...,2015-08-31
3,US,4332923,RRB9C05HDOD4O,B00QUFTPV4,221169481,abcGoodefg® FBI Covert Acoustic Tube Earpiece ...,Mobile_Electronics,4,0,0,N,Y,Work very well but couldn't get used to not he...,Work very well but couldn't get used to not he...,2015-08-31
4,US,44855305,R26I2RI1GFV8QG,B0067XVNTG,563475445,Generic Car Dashboard Video Camera Vehicle Vid...,Mobile_Electronics,2,0,0,N,Y,Cameras has battery issues,"Be careful with these products, I have bought ...",2015-08-31


In [11]:
df["Sentiment"] = df["star_rating"].apply(lambda score: "positive" if score >= 3 else "negative")
df['Sentiment'] = df['Sentiment'].map({'positive':1, 'negative':0})

In [12]:
df = df[["review_body", "Sentiment"]]

In [13]:
# Dropping last n rows using drop
n = 54975
df.drop(df.tail(n).index,
        inplace = True)

In [14]:
index = df.index
number_of_rows = len(index)
print(number_of_rows)

50000


In [15]:
df.tail()

,review_body,Sentiment
49995,I love this product and it is really easy to u...,1
49996,I've have been using HIFONICS since 1994 & I d...,1
49997,Lifestyle Accessory brand 6 Foot Wall Charger ...,0
49998,This charger is not what it looks like. Horrib...,0
49999,I purchased a Sonaverse portable speaker to li...,1


In [16]:
df.head()

,review_body,Sentiment
0,"As advertised. Everything works perfectly, I'm...",1
1,it's great,1
2,These work great and fit my life proof case fo...,1
3,Work very well but couldn't get used to not he...,1
4,"Be careful with these products, I have bought ...",0


In [17]:
reviews = df['review_body'].values.tolist()
labels = df['Sentiment'].tolist()

In [18]:
print(reviews[:2])
print(labels[:2])

["As advertised. Everything works perfectly, I'm very happy with the camera. As a matter of fact I'm going to buy another one for my 2nd car.", "it's great"]
[1, 1]


In [19]:
from sklearn.model_selection import train_test_split
training_sentences, validation_sentences, training_labels, validation_labels = train_test_split(reviews, labels, test_size=.2)


In [20]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [21]:
tokenizer([training_sentences[0]], truncation=True,
                            padding=True, max_length=128)

{'input_ids': [[101, 2023, 2428, 18227, 1996, 8529, 5104, 1012, 2130, 2065, 3333, 1010, 2498, 2097, 4148, 2000, 2068, 1012, 2200, 2200, 5851, 1012, 2036, 2009, 2064, 4906, 2062, 2084, 1019, 2302, 15011, 2151, 1997, 1996, 8529, 5104, 2030, 3638, 12668, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [22]:
def clean_text(text):
  if isinstance(text, str):  # Check if text is already a string
    return text
  elif text is None:  # Check for None values
    return ''
  else:
    return str(text)  # Attempt to convert to string if not None

training_sentences_cleaned = [clean_text(text) for text in training_sentences]
validation_sentences_cleaned = [clean_text(text) for text in validation_sentences]

train_encodings = tokenizer(training_sentences_cleaned, truncation=True, padding=True)
val_encodings = tokenizer(validation_sentences_cleaned, truncation=True, padding=True)

In [23]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))

In [24]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [25]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)

# Instead of using model.compute_loss directly, use the model's built-in loss calculation
def custom_loss_function(labels, logits):
    """
    Custom loss function wrapper.

    Args:
        labels: True labels.
        logits: Model's output logits.

    Returns:
        The computed loss.
    """
    # Ensure labels and logits have the correct shape and type
    labels = tf.cast(labels, tf.int32)  # Explicitly cast labels to int32
    logits = tf.cast(logits, tf.float32)  # Explicitly cast logits to float32

    # Calculate the loss using SparseCategoricalCrossentropy
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    loss = loss_fn(labels, logits)

    return loss

# Compile the model using the custom_loss_function
model.compile(optimizer=optimizer, loss=custom_loss_function, metrics=['accuracy'])
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=2,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16))

Epoch 1/2
2500/2500 [==============================] - 2369s 939ms/step - loss: 0.2137 - accuracy: 0.9121 - val_loss: 0.1913 - val_accuracy: 0.9253
Epoch 2/2
2500/2500 [==============================] - 2361s 944ms/step - loss: 0.1388 - accuracy: 0.9477 - val_loss: 0.2071 - val_accuracy: 0.9275


In [26]:
model.save_pretrained("./sentiment")

In [28]:
loaded_model = TFDistilBertForSequenceClassification.from_pretrained("./sentiment")  # Use relative path to load from the local directory


Some layers from the model checkpoint at ./sentiment were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ./sentiment and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
test_sentence = "This is a really bad product. I hate it"



predict_input = tokenizer.encode(test_sentence,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")

tf_output = loaded_model.predict(predict_input)[0]


tf_prediction = tf.nn.softmax(tf_output, axis=1)
labels = ['Negative','Positive']
label = tf.argmax(tf_prediction, axis=1)
label = label.numpy()
print(labels[label[0]])

1/1 [==============================] - 1s 1s/step
Negative
